In [1]:
from pathlib import Path
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString, MultiLineString
from shapely.ops import unary_union, linemerge

In [2]:
RAIL_SHP = "railshape/NWR_TrackCentreLines.shp" 
OLE_SHP  = "oleshape/oleshape.shp"
DEDUPLICATE = True 
SAVE_CSV = True
OUT_PREFIX = "uk_rail_stock"

In [3]:
INTENSITIES = [
    {"item": "Steel rails",                     "material": "Steel",    "apply_to": "railway", "value": 121.0,  "unit": "t/km"},
    {"item": "Sub-base aggregates",             "material": "Stone",    "apply_to": "railway", "value": 3000.0, "unit": "t/km"},
    {"item": "Pre-cast concrete sleepers – G44","material": "Concrete", "apply_to": "railway", "value": 480.5,  "unit": "t/km"},
    {"item": "Ballast",                         "material": "Stone",    "apply_to": "railway", "value": 4200.0, "unit": "t/km"},
    {"item": "Contact wires (OLE)",             "material": "Copper",   "apply_to": "ole",     "value": 1.914,  "unit": "t/km"},
]

In [4]:
def _is_line_geom(geom):
    return isinstance(geom, (LineString, MultiLineString))

def _assert_epsg27700(gdf, path_str):
    epsg = gdf.crs.to_epsg() if gdf.crs is not None else None
    if epsg != 27700:
        raise ValueError(f"{path_str} is not EPSG:27700（!!：{gdf.crs}）。")

def _total_length_km_27700(shp_path: str, deduplicate: bool = True) -> float:
    p = Path(shp_path) if shp_path else None
    if not p or not p.exists():
        return 0.0
    gdf = gpd.read_file(p)
    _assert_epsg27700(gdf, p.name)

    gdf = gdf[gdf.geometry.apply(_is_line_geom)]
    if gdf.empty:
        return 0.0

    if deduplicate:
        merged = linemerge(unary_union(gdf.geometry))
        if isinstance(merged, LineString):
            length_m = merged.length
        elif isinstance(merged, MultiLineString):
            length_m = sum(ln.length for ln in merged.geoms)
        else:
            length_m = gdf.length.sum()
    else:
        length_m = gdf.length.sum()

    return float(length_m) / 1000.0  # km

def _to_t_per_km(value: float, unit: str) -> float:
    u = (unit or "").strip().lower()
    if u in {"t/km", "tonne/km", "tonnes/km"}:
        return float(value)
    if u in {"kg/m", "kilogram/m"}:
        return float(value) * 1.0  # 1 kg/m == 1 t/km
    raise ValueError(f"not: {unit}（only't/km' or'kg/m'）")


rail_km = _total_length_km_27700(RAIL_SHP, DEDUPLICATE)
ole_km  = _total_length_km_27700(OLE_SHP,  DEDUPLICATE)

lengths_df = pd.DataFrame([
    {"layer": "railway", "length_km": rail_km},
    {"layer": "ole",     "length_km": ole_km},
])


rows = []
for r in INTENSITIES:
    t_per_km = _to_t_per_km(r["value"], r["unit"])
    target = r["apply_to"].strip().lower()
    L = rail_km if target == "railway" else ole_km
    rows.append({
        "item": r["item"],
        "material": r.get("material", ""),
        "apply_to": r["apply_to"],
        "length_km_used": L,
        "intensity_t_per_km": t_per_km,
        "mass_t": L * t_per_km
    })
stock_df = pd.DataFrame(rows)

totals_by_layer = stock_df.groupby("apply_to", as_index=False)["mass_t"] \
                          .sum().rename(columns={"mass_t": "mass_t_total_by_layer"})
totals_by_material = stock_df.groupby("material", as_index=False)["mass_t"] \
                             .sum().rename(columns={"mass_t": "mass_t_total_by_material"})
grand_total_t = float(stock_df["mass_t"].sum())


print(f"Railway length: {rail_km:,.2f} km")
print(f"OLE length:     {ole_km:,.2f} km")
print("\n--- Items ---")
print(stock_df[["item","material","mass_t"]].to_string(index=False))
print("\n--- Totals by layer ---")
print(totals_by_layer.to_string(index=False))
print("\n--- Totals by material ---")
print(totals_by_material.to_string(index=False))
print("\n=== Grand total mass ===")
print(f"{grand_total_t:,.3f} t")

if SAVE_CSV:
    lengths_df.to_csv(f"{OUT_PREFIX}_lengths.csv", index=False)
    stock_df.to_csv(f"{OUT_PREFIX}_stock_items.csv", index=False)
    totals_by_layer.to_csv(f"{OUT_PREFIX}_stock_totals_by_layer.csv", index=False)
    totals_by_material.to_csv(f"{OUT_PREFIX}_stock_totals_by_material.csv", index=False)
    pd.DataFrame([{"grand_total_t": grand_total_t}]).to_csv(
        f"{OUT_PREFIX}_grand_total.csv", index=False
    )

Railway length: 35,924.43 km
OLE length:     19,514.52 km

--- Items ---
                            item material       mass_t
                     Steel rails    Steel 4.346856e+06
             Sub-base aggregates    Stone 1.077733e+08
Pre-cast concrete sleepers – G44 Concrete 1.726169e+07
                         Ballast    Stone 1.508826e+08
             Contact wires (OLE)   Copper 3.735079e+04

--- Totals by layer ---
apply_to  mass_t_total_by_layer
     ole           3.735079e+04
 railway           2.802644e+08

--- Totals by material ---
material  mass_t_total_by_material
Concrete              1.726169e+07
  Copper              3.735079e+04
   Steel              4.346856e+06
   Stone              2.586559e+08

=== Grand total mass ===
280,301,764.768 t
